In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

```{autolink-concat}
```

# Spin alignment

In [ ]:
import logging

import graphviz
import qrules
import sympy as sp
from IPython.display import Math

import ampform
from ampform.helicity import formulate_wigner_d

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)


def show_transition(transition, **kwargs):
    if "size" not in kwargs:
        kwargs["size"] = 5
    dot = qrules.io.asdot(transition, **kwargs)
    display(graphviz.Source(dot))

## Helicity formalism

Imagine we want to formulate the amplitude for the following **single** {class}`~qrules.transition.StateTransition`:

In [ ]:
full_reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["K0", ("Sigma+", [+0.5]), ("p~", [+0.5])],
    allowed_intermediate_particles=["Sigma(1660)~-", "N(1650)+"],
    allowed_interaction_types="strong",
    formalism="helicity",
)
graphs = full_reaction.to_graphs()
single_transition_reaction = full_reaction.from_graphs(
    [graphs[0]], formalism=full_reaction.formalism
)
transition = single_transition_reaction.transitions[0]
show_transition(transition)

The specific {attr}`~qrules.transition.State.spin_projection`s for each {attr}`~qrules.transition.State.particle` only make sense _given a specific reference frame_. AmpForm's {class}`.HelicityAmplitudeBuilder` interprets these projections as the **helicity** $\lambda=\vec{S}\cdot\vec{p}$ of each particle _in the rest frame of the parent particle_. For example, the helicity $\lambda_2=+\tfrac{1}{2}$ of $\bar p$ is the helicity as measured in the rest frame of resonance $\bar\Sigma(1660)^-$. The reason is that these helicities needed when formulating the two-particle state for the decay node $\bar\Sigma(1660)^- \to K^0\bar p$ (see {doc}`/usage/helicity/formalism` and {func}`.formulate_wigner_d`).

Ignoring dynamics and coefficients, the {class}`.HelicityModel` for this single transition is rather simple:

In [ ]:
builder = ampform.get_builder(single_transition_reaction)
model = builder.formulate()
model.expression.subs(model.parameter_defaults).subs(1.0, 1)

The two Wigner-$D$ functions come from the two **two-body decay nodes** that appear in the {class}`~qrules.transition.StateTransition` above. They were formulated with {func}`.formulate_wigner_d`:

In [ ]:
sp.Mul(
    formulate_wigner_d(transition, node_id=0),
    formulate_wigner_d(transition, node_id=1),
)

Now, as {func}`.formulate_wigner_d` explains, the numbers that appear in the Wigner-$D$ functions here are computed from the helicities of the decay products. But there's a subtle problem with this: these helicities are _assumed to be in the rest frame of each parent particle_. For the first node, this is fine, because the parent particle rest frame matches that of the initial state in the {class}`~qrules.transition.StateTransition` above. In the second node, however, we are in a different rest frame.

When summing over all amplitudes in the complete {class}`~qrules.transition.StateTransitionCollection` that contains all spin projections (helicities), the mismatch in rest frames evens out and the problem we identified here can be ignored. It again becomes a problem, however, when we are formulating an amplitude model _with different topologies_. An example would be the following reaction:

In [ ]:
show_transition(full_reaction, collapse_graphs=True)

<!-- cspell:ignore mikhasenko Dalitzplot Threebody -->
The {class}`.HelicityAmplitudeBuilder` implements the 'standard' helicity formalism as described in {cite}`richmanExperimenterGuideHelicity1984, kutschkeAngularDistributionCookbook1996, chungSpinFormalismsUpdated2014` and simply sums over the different amplitudes to get the full amplitude:

In [ ]:
builder = ampform.get_builder(full_reaction)
model = builder.formulate()
latex = sp.multiline_latex(
    sp.Symbol("I"),
    model.expression.subs(model.parameter_defaults).subs(1.0, 1),
    environment="eqnarray",
)
Math(latex)

As pointed out in {cite}`marangottoHelicityAmplitudesGeneric2020, mikhasenkoDalitzplotDecompositionThreebody2020, wangNovelMethodTest2020`, this is wrong because of the mismatch in reference frames for the helicities.

## Aligning reference frames

In what follows, we follow {cite}`marangottoHelicityAmplitudesGeneric2020` to align all amplitudes in the different topologies back to the initial state reference frame $A$, so that they can be correctly summed up. Specifically, we want to formulate a new, correctly aligned amplitude $\mathcal{A}^{A\to 0,1,\dots}_{m_A,m_0,m_1,\dots}$ from the original amplitudes $\mathcal{A}^{A\to R,S,i,...\to 0,1,\dots}_{\lambda_A,\lambda_0,\lambda_1,\dots}$ by applying Eq.(45) and Eq.(47) for generic, multi-body decays.

In the following, we test the implementation with 1-to-3 body decays, just as in {cite}`marangottoHelicityAmplitudesGeneric2020`. We use the notation from {func}`.get_boost_chain_suffix` to indicate resonances $R,S,U$:

In [ ]:
dot1 = """
digraph {
    bgcolor=none
    rankdir=LR
    edge [arrowhead=none]
    node [shape=none, width=0]
    A
    0 [fontcolor=red]
    1 [fontcolor=green, label=<<o>1</o>>]
    2 [fontcolor=blue, label=<<o>2</o>>]
    { rank=same A }
    { rank=same 0, 1, 2 }
    N0 [label=""]
    N1 [label=""]
    A -> N0 [style=dotted]
    N0 -> N1 [label="R = 01", fontcolor=orange]
    N1 -> 0
    N0 -> 2 [style=dashed]
    N1 -> 1 [style=dashed]
}
"""
dot2 = """
digraph {
    bgcolor=none
    rankdir=LR
    edge [arrowhead=none]
    node [shape=none, width=0]
    A
    0 [label=0, fontcolor=red]
    1 [label=1, fontcolor=green, label=<<o>1</o>>]
    2 [label=2, fontcolor=blue, label=<<o>2</o>>]
    { rank=same A }
    { rank=same 0, 1, 2 }
    N0 [label=""]
    N1 [label=""]
    A -> N0 [style=dotted]
    N0 -> N1 [label="S = 02", fontcolor=violet]
    N1 -> 0
    N0 -> 1 [style=dashed]
    N1 -> 2 [style=dashed]
}
"""
display(*map(graphviz.Source, [dot1, dot2]))

Rewriting and recoloring Equations (45) and (46) from {cite}`marangottoHelicityAmplitudesGeneric2020`:

$$
\begin{eqnarray}
\mathcal{A}^{A \to {\color{orange}R},2 \to 0,1,2}_{m_A,m_0,m_1,m_2}
&=&
  \sum_{\lambda_0^{01},\mu_0^{01},\nu_0^{01}}
    {\color{red}{D^{s_0}_{m_0,\nu_0^{01}}}}\!\left({\color{red}{\alpha_0^{01}, \beta_0^{01}, \gamma_0^{01}}}\right)
      {\color{red}{D^{s_0}_{\nu_0^{01},\mu_0^{01}}}}\!\left({\color{orange}{\phi_{_{01}}, \theta_{_{01}}}}, 0\right)
      {\color{red}{D^{s_0}_{\mu_0^{01},\lambda_0^{01}}}}\!\left({\color{red}{\phi_0^{01}, \theta_0^{01}}}\right) \\
&\times&
  \sum_{\lambda_1^{01},\mu_1^{01},\nu_1^{01}}
    {\color{green}{D^{s_1}_{m_1,\nu_1^{01}}}}\!\left({\color{green}{\alpha_1^{01}, \beta_1^{01}, \gamma_1^{01}}}\right)
      {\color{green}{D^{s_1}_{\nu_1^{01},\mu_1^{01}}}}\!\left({\color{orange}{\phi_{_{01}}, \theta_{_{01}}}}, 0\right)
      {\color{green}{D^{s_1}_{\mu_1^{01},\lambda_1^{01}}}}\!\left({\color{red}{\phi_0^{01}, \theta_0^{01}}}\right) \\
&\times&
  \sum_{\lambda_2^{01}}
    {\color{blue}{D^{s_2}_{m_2,\lambda_2^{01}}}}\!\left({\color{orange}{\phi_{_{01}}, \theta_{_{01}}}}, 0\right) \\
&\times&
  \mathcal{A}^{A \to {\color{orange}R},2 \to 0,1,2}_{m_A,\lambda_0^{01},\bar\lambda_1^{01},\bar\lambda_2^{01}}
\end{eqnarray}
$$ (alignment-R)

$$
\begin{eqnarray}
\mathcal{A}^{A \to {\color{violet}S},1 \to 0,1,2}_{m_A,m_0,m_1,m_2}
&=&
  \sum_{\lambda_0^{02},\mu_0^{02},\nu_0^{02}}
    {\color{red}{D^{s_0}_{m_0,\nu_0^{02}}}}\!\left({\color{red}{\alpha_0^{02}, \beta_0^{02}, \gamma_0^{02}}}\right)
      {\color{red}{D^{s_0}_{\nu_0^{02},\mu_0^{02}}}}\!\left({\color{violet}{\phi_{_{02}}, \theta_{_{02}}}}, 0\right)
      {\color{red}{D^{s_0}_{\mu_0^{02},\lambda_0^{02}}}}\!\left({\color{red}{\phi_0^{02}, \theta_0^{02}}}\right) \\
&\times&
  \sum_{\lambda_1^{02}}
    {\color{green}{D^{s_1}_{m_1,\lambda_1^{02}}}}\!\left({\color{violet}{\phi_{_{02}}, \theta_{_{02}}}}, 0\right) \\
&\times&
  \sum_{\lambda_2^{02},\mu_2^{02},\nu_2^{02}}
    {\color{blue}{D^{s_2}_{m_2,\nu_2^{02}}}}\!\left({\color{blue}{\alpha_2^{02}, \beta_2^{02}, \gamma_2^{02}}}\right)
      {\color{blue}{D^{s_2}_{\nu_2^{02},\mu_2^{02}}}}\!\left({\color{violet}{\phi_{_{02}}, \theta_{_{02}}}}, 0\right)
      {\color{blue}{D^{s_2}_{\mu_2^{02},\lambda_2^{02}}}}\!\left({\color{red}{\phi_0^{02}, \theta_0^{02}}}\right) \\
&\times&
  \mathcal{A}^{A \to {\color{violet}S},2 \to 0,1,2}_{m_A,\lambda_0^{02},\bar\lambda_1^{02},\bar\lambda_2^{02}}
\end{eqnarray}
$$ (alignment-S)

:::{note}

What should the alignment summation in $\mathcal{A}^{A \to {\color{turquoise}U},0 \to 0,1,2}_{m_A,m_0,m_1,m_2}$ look like?

:::

In [ ]:
dot3 = """
digraph {
    bgcolor=none
    rankdir=LR
    edge [arrowhead=none]
    node [shape=none, width=0]
    0 [shape=none, label=0, fontcolor=red]
    1 [shape=none, label=1, fontcolor=green]
    2 [shape=none, label=2, fontcolor=blue, label=<<o>2</o>>]
    A [shape=none, label=A]
    { rank=same A }
    { rank=same 0, 1, 2 }
    N0 [label=""]
    N1 [label=""]
    A -> N0 [style=dotted]
    N0 -> N1 [label=<U =<o>12</o>>, fontcolor=turquoise, style=dashed]
    N0 -> 0
    N1 -> 1
    N1 -> 2 [style=dashed]
}
"""
graphviz.Source(dot3)

$$
\begin{eqnarray}
\mathcal{A}^{A \to {\color{turquoise}U},0 \to 0,1,2}_{m_A,m_0,m_1,m_2}
&=&
  \sum_{\lambda_0^{12}}
    {\color{red}{D^{s_0}_{m_0,\lambda_0^{12}}}}\!\left({\color{red}{\phi_0, \theta_0}}, 0\right) \\
&\times&
  \sum_{\lambda_1^{12},\mu_1^{12},\nu_1^{12}}
    {\color{green}{D^{s_0}_{m_0,\nu_1^{12}}}}\!\left({\color{green}{\alpha_1^{12}, \beta_1^{12}, \gamma_1^{12}}}\right)
      {\color{green}{D^{s_0}_{\nu_1^{12},\mu_1^{12}}}}\!\left({\color{red}{\phi_0, \theta_0}}, 0\right)
      {\color{green}{D^{s_0}_{\mu_1^{12},\lambda_1^{12}}}}\!\left({\color{green}{\phi_1^{12}, \theta_1^{12}}}\right) \\
&\times&
  \sum_{\lambda_2^{12},\mu_2^{12},\nu_2^{12}}
    {\color{blue}{D^{s_2}_{m_2,\nu_2^{12}}}}\!\left({\color{blue}{\alpha_2^{12}, \beta_2^{12}, \gamma_2^{12}}}\right)
      {\color{blue}{D^{s_2}_{\nu_2^{12},\mu_2^{12}}}}\!\left({\color{red}{\phi_0, \theta_0}}, 0\right)
      {\color{blue}{D^{s_2}_{\mu_2^{12},\lambda_2^{12}}}}\!\left({\color{green}{\phi_1^{12}, \theta_1^{12}}}\right) \\
&\times&
  \mathcal{A}^{A \to {\color{turquoise}S},2 \to 0,1,2}_{m_A,\lambda_1^{12},\bar\lambda_1^{12},\bar\lambda_2^{12}}
\end{eqnarray}
$$ (alignment-U)

### $J/\psi \to K^0 \Sigma^+ \bar{p}$

In [ ]:
from ampform.helicity import (
    formulate_helicity_rotation_chain,
    formulate_rotation_chain,
    formulate_spin_alignment,
)


def show_all_spin_matrices(transition, functor, cleanup: bool) -> None:
    for i in transition.final_states:
        state = transition.states[i]
        particle_name = state.particle.latex
        s = sp.Rational(state.particle.spin)
        m = sp.Rational(state.spin_projection)
        display(
            Math(
                Rf"|s_{i},m_{i}\rangle=|{s},{m}\rangle \quad ({particle_name})"
            )
        )
        summation = functor(transition, i)
        if cleanup:
            summation = summation.cleanup()
        display(summation)

We pick one transition and want to formulate the Wigner-$D$ functions that appear in Eq.(45). The first step is to use {func}`.formulate_helicity_rotation_chain` to generate all Wigner-$D$ **helicity rotations** (see {func}`.formulate_helicity_rotation`) for each final state. These helicity rotations "undo" all rotations that came from each Lorentz boosts when boosting from initial state $J/\psi$ to each final state:

In [ ]:
transition_r = full_reaction.transitions[-1]
show_transition(transition_r)
show_all_spin_matrices(
    transition_r, formulate_helicity_rotation_chain, cleanup=True
)

The function {func}`.formulate_rotation_chain` goes one step further. It adds a **Wigner rotation** (see {func}`.formulate_wigner_rotation`) to the generated list of helicity rotation Wigner-$D$ functions in case there are resonances in between the initial state and rotated final state. If there are no resonances in between (here, state `2`, the $\bar p$), there is only one helicity rotation and there is no need for a Wigner rotation.

In [ ]:
show_all_spin_matrices(transition_r, formulate_rotation_chain, cleanup=False)

**These are indeed all the terms that we see in Equation {eq}`alignment-R`!**

To create all sum combinations for all final states, we can use {func}`.formulate_spin_alignment`. This should give the sum of Eq.(45):

In [ ]:
alignment_summation = formulate_spin_alignment(transition_r)
alignment_summation.cleanup()

Finally, here are the generated spin alignment terms for the other two decay chains. Notice that the first is indeed the same as {eq}`alignment-S`:

In [ ]:
reaction_s = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["K0", ("Sigma+", [+0.5]), ("p~", [+0.5])],
    allowed_intermediate_particles=["N(1650)+"],
    allowed_interaction_types="strong",
    formalism="helicity",
)
transition_s = reaction_s.transitions[0]
show_transition(transition_s)
show_all_spin_matrices(transition_s, formulate_rotation_chain, cleanup=False)

...and that the second matches Equation {eq}`alignment-U`:

In [ ]:
reaction_u = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["K0", ("Sigma+", [+0.5]), ("p~", [+0.5])],
    allowed_intermediate_particles=["K*(1680)~0"],
    allowed_interaction_types="strong",
    formalism="helicity",
)
transition_u = reaction_u.transitions[0]
show_transition(transition_u)
show_all_spin_matrices(transition_u, formulate_rotation_chain, cleanup=False)

## Compute Wigner rotation angles

In [ ]:
from ampform.kinematics import (
    compute_boost_chain,
    create_four_momentum_symbols,
)

dot = qrules.io.asdot(transition_u)
topology = transition_u.topology
display(graphviz.Source(dot))
momenta = create_four_momentum_symbols(topology)
for state_id in topology.outgoing_edge_ids:
    boosts = compute_boost_chain(topology, momenta, state_id)
    display(sp.Array(boosts))

In [ ]:
from ampform.kinematics import compute_wigner_rotation_matrix

for state_id in topology.outgoing_edge_ids:
    expr = compute_wigner_rotation_matrix(topology, momenta, state_id)
    display(expr)

In [ ]:
from ampform.kinematics import compute_wigner_angles

angles = {}
for state_id in topology.outgoing_edge_ids:
    angle_definitions = compute_wigner_angles(topology, momenta, state_id)
    for name, expr in angle_definitions.items():
        angle_symbol = sp.Symbol(name, real=True)
        angles[angle_symbol] = expr
        latex = sp.multiline_latex(angle_symbol, expr, environment="eqnarray")
        display(Math(latex))

### Lambdification

:::{error}

Lambdification of the above expression currently results in horrifically long source code. This will cause problems in [TensorWaves](https://tensorwaves.rtfd.io).
{{ "See generated NumPy code for this angle {doc}`here <generated-code>`." if EXECUTE_NB else "" }}

:::

In [ ]:
import inspect

beta = sp.Symbol("beta_1^12", real=True)
beta_expr = angles[beta]

func = sp.lambdify(momenta.values(), beta_expr, cse=True)
src = inspect.getsource(func)
n_characters = len(src)
latex = sp.latex(beta)
latex += Rf":\quad\text{{{n_characters:,} characters in generated code}}"
Math(latex)

{{ "```{toctree}\n:hidden:\ngenerated-code\n```" if EXECUTE_NB else "" }}